In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
import lxml
import pandas as pd
import requests
import yes24_dbio as ydb

In [2]:
# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

In [3]:
data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller20081_202312_1000_v2.csv")
urls = data2['detail_link']
urls = list(set(urls))
len(urls)

C:\Users\hi\AppData\Local\Temp\ipykernel_16588\1253186524.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_csv(f"./data/yes24/new_yes24bestseller20081_202312_1000_v2.csv")


42780

In [4]:
# driver.get(urls[0])
# driver.find_element(By.CSS_SELECTOR, "#txtOneCommentCount").text.strip("()건")  # 한줄평 총 갯수

In [5]:
# driver.find_element(By.CSS_SELECTOR, "#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_reviewCount > a").click()

In [23]:
167+31+13+82

293

In [ ]:
for idx, d_url in enumerate(urls[293:]):
    book_id = d_url.split("/")[-1] # 책 id
    driver.get(d_url)
    driver.find_element(By.CSS_SELECTOR, "#txtOneCommentCount").text.strip("()건")  # 한줄평 총 갯수
    #     driver.find_element(By.CSS_SELECTOR, "#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_reviewCount > a").click()
    driver.find_element(By.CSS_SELECTOR, "#yDetailTabNavWrap > div > div.gd_tabBar > ul > li:nth-child(3) > a").click()
    
    # 마우스 스크롤 올리기
    script = f"window.scrollTo({9000},{8000})"  # 자바 스크립트로 위치 지정
    driver.execute_script(script)         # 스크롤 실행
    time.sleep(2)
        
    try:
        driver.find_element(By.CSS_SELECTOR, "#commit_recent").click()
#         driver.find_element(By.CSS_SELECTOR, ".bgYUI.first.dim").click()
    except Exception as e :
        pass
    soup = bs(driver.page_source, 'lxml')
    # 한줄평 갯수
    n_one_line_review = int(soup.select_one("#txtOneCommentCount").text.strip("()건"))  # 한줄평 총 갯수

    total_one_line_reviews = []

    if n_one_line_review == 0:
        pass
    elif n_one_line_review > 0:
        n_page_one_line_review = int(n_one_line_review) // 6 + 1  # 한줄평 총 페이지 수
        print(f"{idx}/{len(urls)}, 한줄평 총 갯수: {n_one_line_review}, 한줄평 총 페이지 수{n_page_one_line_review}", end="\r")
        if n_page_one_line_review == 1:
            total_one_line_reviews += [item.text for item in soup.select("div.cmt_cont > span.txt")]
        elif n_page_one_line_review > 1:
#             driver.find_element(By.CSS_SELECTOR, "#infoset_reviewContentList > div.review_sort.sortTop > div.review_sortLft > div > a:nth-child(4)").click() 
            soup2 = bs(driver.page_source, 'lxml')
            total_one_line_reviews += [item.text for item in soup2.select("div.cmt_cont > span.txt")]
            page_num_link = soup2.select_one("#infoset_oneCommentList > div:nth-child(2) > div.rvCmt_sortLft > div > a:nth-child(4)")['href'][:-1]
            for p in range(2, n_page_one_line_review+1):
                r = requests.get(f"https://www.yes24.com/{page_num_link}{p}")
                print(f"{idx}/{len(urls)}에서 {p}/{n_page_one_line_review+1}처리중", end="\r")
                time.sleep(5)
                soup3 = bs(r.text, 'lxml')
                total_one_line_reviews += [item.text for item in soup3.select("div.cmt_cont > span.txt")]
    line_review_result = pd.DataFrame({"book_id" : book_id, "one_line_review" : total_one_line_reviews})
    line_review_result.to_csv(f"./data/yes24/one_line_reviews/{book_id}_one_line_reviews.csv", index=False)
    ydb.to_mysql(line_review_result, "yes24best", "yes24_one_line_reviews")

In [ ]:
line_review_result.info()

In [22]:
print(idx)

82
